In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['conversiones', 'content_category_top', 'content_category_bottom', 'samplesubmission', 'content_category', 'site_id', 'page', 'pageviews', 'device_data']


In [2]:
data = pd.read_csv("../input/pageviews/pageviews.csv",
                   parse_dates=["FEC_EVENT"])
data

,FEC_EVENT,PAGE,CONTENT_CATEGORY,CONTENT_CATEGORY_TOP,CONTENT_CATEGORY_BOTTOM,SITE_ID,ON_SITE_SEARCH_TERM,USER_ID
0,2018-03-30 07:35:48,1,1,1,1,1,1,0
1,2018-03-30 07:35:52,2,2,2,2,2,1,0
2,2018-03-30 07:36:11,3,2,2,2,3,1,0
3,2018-03-30 07:36:16,4,2,2,2,3,1,0
4,2018-03-30 07:41:38,5,2,2,2,2,1,0
5,2018-03-30 07:41:42,2,2,2,2,2,1,0
6,2018-03-30 07:42:01,3,2,2,2,3,1,0
7,2018-03-30 07:42:05,4,2,2,2,3,1,0
8,2018-03-30 07:43:43,3,2,2,2,3,1,0
9,2018-03-30 07:44:14,6,2,2,2,3,1,0


In [3]:
X_test = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_test.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_test = pd.concat(X_test, axis=1)

haciendo PAGE
haciendo CONTENT_CATEGORY
haciendo CONTENT_CATEGORY_TOP
haciendo CONTENT_CATEGORY_BOTTOM
haciendo SITE_ID
haciendo ON_SITE_SEARCH_TERM


In [4]:
data = data[data.FEC_EVENT.dt.month < 10]
X_train = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_train.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_train = pd.concat(X_train, axis=1)

haciendo PAGE
haciendo CONTENT_CATEGORY
haciendo CONTENT_CATEGORY_TOP
haciendo CONTENT_CATEGORY_BOTTOM
haciendo SITE_ID
haciendo ON_SITE_SEARCH_TERM


In [5]:
features = list(set(X_train.columns).intersection(set(X_test.columns)))
X_train = X_train[features]
X_test = X_test[features]

In [6]:
y_prev = pd.read_csv("../input/conversiones/conversiones.csv")
y_train = pd.Series(0, index=X_train.index)
idx = set(y_prev[y_prev.mes >= 10].USER_ID.unique()).intersection(
        set(X_train.index))
y_train.loc[list(idx)] = 1

In [7]:
#import the library
from sklearn import model_selection
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from termcolor import colored
import warnings
warnings.filterwarnings('ignore')


fi = []
test_probs = []
i = 0
for train_idx, valid_idx in model_selection.KFold(n_splits=20, shuffle=True).split(X_train):
    i += 1
    Xt = X_train.iloc[train_idx]
    yt = y_train.loc[X_train.index].iloc[train_idx]

    Xv = X_train.iloc[valid_idx]
    yv = y_train.loc[X_train.index].iloc[valid_idx]
    
    #dt = xgb.DMatrix(Xt.as_matrix(),label=yt.as_matrix())
    #dv = xgb.DMatrix(Xv.as_matrix(),label=yv.as_matrix())
    
    params = {
    "booster": 'gbtree',
    "reg_lambda" : 2,
    "eta": 0.1,
    "max_depth": 3,
    "objective": "binary:logistic",
    "silent": 0,
    "base_score": np.mean(yt),
    'n_estimators': 1000,
    "eval_metric": "auc"}
    
    learner = xgb.XGBClassifier(n_estimators=1000)
    eval_set  = [(Xt,yt), (Xv,yv)]
    learner.fit(Xt, yt, eval_set=eval_set, eval_metric="auc", early_stopping_rounds=30)
     
    test_probs.append(pd.Series(learner.predict_proba(X_test)[:, -1],
                                index=X_test.index, name="fold_" + str(i)))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))

test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
test_probs.index.name="USER_ID"
test_probs.name="SCORE"
test_probs.to_csv("benchmark", header=True, compression="zip")
fi = pd.concat(fi, axis=1).mean(axis=1)

[0]	validation_0-auc:0.768119	validation_1-auc:0.831171
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 30 rounds.
[1]	validation_0-auc:0.774031	validation_1-auc:0.838053
[2]	validation_0-auc:0.777972	validation_1-auc:0.838335
[3]	validation_0-auc:0.777681	validation_1-auc:0.839012
[4]	validation_0-auc:0.784536	validation_1-auc:0.850801
[5]	validation_0-auc:0.830546	validation_1-auc:0.913865
[6]	validation_0-auc:0.836238	validation_1-auc:0.911157
[7]	validation_0-auc:0.836581	validation_1-auc:0.910819
[8]	validation_0-auc:0.838451	validation_1-auc:0.908958
[9]	validation_0-auc:0.840624	validation_1-auc:0.913301
[10]	validation_0-auc:0.844986	validation_1-auc:0.912793
[11]	validation_0-auc:0.847177	validation_1-auc:0.913132
[12]	validation_0-auc:0.851396	validation_1-auc:0.91285
[13]	validation_0-auc:0.855784	validation_1-auc:0.913752
[14]	validation_0-auc:0.858453	validation_1-auc:0.912116